자동차 브랜드 정보 제공 검색 서비스

라이브러리 import, install

In [ ]:
#준비
from google.colab import drive
drive.mount('/content/drive/')

import pandas as pd
import numpy as np

import os
import sys
import urllib.request
import json
import re

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# 라이브러리 설치
!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev python3-dev
!pip3 install JPype1-py3
!pip3 install konlpy
# JAVA_HOME 환경변수 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64" 

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:11 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Fetched 340 kB in 1s (272 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'python-dev-is-pyt

In [ ]:
import nltk

from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt') # tokenizer
from nltk.corpus import stopwords

from konlpy.tag import Okt
okt = Okt()

from collections import Counter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


데이터 수집

Naver Cafe 검색 API

In [ ]:
#사용자가 입력한 검색 query로 카페글 검색 API를 불러오는 함수 정의 -> 검색한 정보에 대한 검색 결과 데이터 프레임 반환
def naver_cafe(input_word):
  client_id = "Bwq4GJAba1UhQUSG9RDd"
  client_secret = "0qM9a1vSPt"

  query = urllib.parse.quote(input_word)

  idx = 0       # 여러개 정보 받을 때 횟수 카운트 용 
  display = 10 # 10개 단위로 정보 받아오기
  start = 1     # 첫 시작점
  end = 1000    # 마지막 시작점

  cafe_df = pd.DataFrame(columns=('Title', 'Link', 'Description', 'Cafe Name', 'Cafe URL'))

  for start_index in range(start, end, display):

    url = "https://openapi.naver.com/v1/search/cafearticle?query=" + query \
          + '&display=' + str(display) \
          + '&start=' + str(start_index)

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200): # API에 정상적인 응답을 받은 경우(데이터 크롤링에 성공한 경우)
        response_body = response.read()
        response_dict = json.loads(response_body.decode('utf-8'))
        items = response_dict['items'] # 원하는 정보가 담겨있는 dict
        for item_index in range(0, len(items)):
          remove_tag = re.compile('<.*?>')
          
          title = re.sub(remove_tag, '', items[item_index]['title'])
          link = items[item_index]['link']
          description = re.sub(remove_tag, '', items[item_index]['description'])
          cafename = items[item_index]['cafename']
          cafeurl = items[item_index]['cafeurl']

          cafe_df.loc[idx] = [title, link, description, cafename, cafeurl]
          idx += 1
    else:
        print("Error Code:" + rescode)

  return cafe_df

Naver News 검색 API

In [ ]:
#사용자가 입력한 검색 query로 뉴스 검색 API를 불러오는 함수 정의 -> 검색한 정보에 대한 검색 결과 데이터 프레임 반환
def naver_news(input_word):
  client_id = "Bwq4GJAba1UhQUSG9RDd"
  client_secret = "0qM9a1vSPt"

  query = urllib.parse.quote(input_word)

  idx = 0       # 여러개 정보 받을 때 횟수 카운트 용 
  display = 10 # 10개 단위로 정보 받아오기
  start = 1     # 첫 시작점
  end = 1000    # 마지막 시작점

  news_df = pd.DataFrame(columns=('Title', 'Link', 'desc', 'date'))

  for start_index in range(start, end, display):

    url = "https://openapi.naver.com/v1/search/news?query=" + query \
          + '&display=' + str(display) \
          + '&start=' + str(start_index)

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200): # API에 정상적인 응답을 받은 경우(데이터 크롤링에 성공한 경우)
        response_body = response.read()
        response_dict = json.loads(response_body.decode('utf-8'))
        items = response_dict['items'] # 원하는 정보가 담겨있는 dict
        for item_index in range(0, len(items)):
          #수정할 부분
          remove_tag = re.compile('<.*?>')
          title = re.sub(remove_tag, '', items[item_index]['title'])
          link = items[item_index]['link']
          org_link = items[item_index]['originallink']
          desc = items[item_index]['description']
          date = items[item_index]['pubDate']

          news_df.loc[idx] = [title, link, desc, date]
          idx += 1
    else:
        print("Error Code:" + rescode)

  return news_df

Naver Blog 검색 API

In [ ]:
#사용자가 입력한 검색 query로 블로그 검색 API를 불러오는 함수 정의 -> 검색한 정보에 대한 검색 결과 데이터 프레임 반환
def naver_blog(input_word):
  client_id = "Bwq4GJAba1UhQUSG9RDd"
  client_secret = "0qM9a1vSPt"

  query = urllib.parse.quote(input_word)

  idx = 0       # 여러개 정보 받을 때 횟수 카운트 용 
  display = 10 # 10개 단위로 정보 받아오기
  start = 1     # 첫 시작점
  end = 1000    # 마지막 시작점

  blog_df = pd.DataFrame(columns=('Title', 'Link', 'Description', 'Blogger Name', 'Blogger Link'))

  for start_index in range(start, end, display):

    url = "https://openapi.naver.com/v1/search/blog?query=" + query \
          + '&display=' + str(display) \
          + '&start=' + str(start_index)

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200): # API에 정상적인 응답을 받은 경우(데이터 크롤링에 성공한 경우)
        response_body = response.read()
        response_dict = json.loads(response_body.decode('utf-8'))
        items = response_dict['items'] # 원하는 정보가 담겨있는 dict
        for item_index in range(0, len(items)):
          remove_tag = re.compile('<.*?>')
          
          title = re.sub(remove_tag, '', items[item_index]['title'])
          link = items[item_index]['link']
          description = re.sub(remove_tag, '', items[item_index]['description'])
          blogger_name = items[item_index]['bloggername']
          blogger_link = items[item_index]['bloggerlink']

          blog_df.loc[idx] = [title, link, description, blogger_name, blogger_link]
          idx += 1
    else:
        print("Error Code:" + rescode)
  
  return blog_df

카페, 뉴스, 블로그 내용 전처리

In [ ]:
#한국어 텍스트 전처리 함수 정의
def clean_korean_text(raw_text):
  #불용어 리스트 불러오기
  with open('/content/drive/MyDrive/Colab Notebooks/stopword_car.txt', 'r', encoding='utf8') as f:
    stopwords_car = f.read().splitlines()
  
  #불용어 제거한 리스트 저장할 리스트 생성
  removed_stopwords_l = []
  #저장된 리스트 안의 문장 단위 탐색
  for sent in raw_text:
    #형태소 단위로 분석
    for_stopwords_list = okt.morphs(sent)
    #불용어 제거하여 저장할 리스트 생성
    word_non_stop = []
    #문장 안의 하나의 단어에 대해서
    for word in for_stopwords_list:
      #불용어 목록에 없다면
      if word not in stopwords_car:
        #제거한 리스트에 저장
        word_non_stop.append(word)
    #중첩 리스트 형태로 문장 단위로 저장
    removed_stopwords_l.append(word_non_stop)

  #형태소 분석
  #형태소 분석하여 저장할 리스트 생성
  clean_words = []
  #각 문장에 대해서
  for text in removed_stopwords_l:
    #pos tagging
    tagged_words = okt.pos(' '.join(text))
    #명사, 동사, 형용사에 해당하는 단어만 리스트에 저장 
    selected_words = [word for word, pos in tagged_words if pos in ['Noun', 'Verb', 'Adjective']]
    #중첩 리스트 형태로 문장 단위 저장
    clean_words.append(selected_words)
  
  return clean_words

In [ ]:
#카페 데이터 프레임 -> 리스트로 저장, 전처리
def cafe_process_api(cafe_df):
  #dataframe에서 텍스트 칼럼만 추출
  cafe_df_drop = cafe_df.drop(['Link', 'Cafe Name', 'Cafe URL'], axis = 1) 
  #텍스트 저장을 위한 빈 리스트 생성
  cafe_texts = []

  for column in ['Title', 'Description']:
    #토큰화, 어간 추출 및 원형 복원
    cafe_df_drop[column] = cafe_df_drop[column].apply(lambda x: ' '.join(okt.morphs(x)))
    #하나의 문자열로 텍스트 저장
    cafe_texts += cafe_df_drop[column].tolist()

  #전처리
  cafe_cleaned_text = clean_korean_text(cafe_texts)

  return cafe_cleaned_text

In [ ]:
#뉴스 데이터 프레임 -> 리스트로 저장, 전처리
def news_process_api(news_df):
  #dataframe에서 텍스트 칼럼만 추출
  news_df_drop = news_df.drop(['Link', 'date'], axis = 1) 
  #텍스트 저장을 위한 빈 리스트 생성
  news_texts = []

  for column in ['Title', 'desc']:
    #토큰화, 어간 추출 및 원형 복원
    news_df_drop[column] = news_df_drop[column].apply(lambda x: ' '.join(okt.morphs(x)))
    #하나의 문자열로 텍스트 저장
    news_texts += news_df_drop[column].tolist()

  #전처리
  news_cleaned_text = clean_korean_text(news_texts)

  return news_cleaned_text

In [ ]:
#블로그 데이터 프레임 -> 리스트로 저장, 전처리
def blog_process_api(blog_df):
  #dataframe에서 텍스트 칼럼만 추출
  blog_df_drop = blog_df.drop(['Link', 'Blogger Name', 'Blogger Link'], axis = 1) 
  #텍스트 저장을 위한 빈 리스트 생성
  blog_texts = []

  for column in ['Title', 'Description']:
    #토큰화, 어간 추출 및 원형 복원
    blog_df_drop[column] = blog_df_drop[column].apply(lambda x: ' '.join(okt.morphs(x)))
    #하나의 문자열로 텍스트 저장
    blog_texts += blog_df_drop[column].tolist()

  #전처리
  blog_cleaned_text = clean_korean_text(blog_texts)

  return blog_cleaned_text

데이터 분석

In [ ]:
#상위 빈도수의 단어와 빈도수 추출하는 함수
def word_count(df_text, brand_name, search_word):
  #결과에서 검색어를 제외하기 위해 검색어를 문자열, 리트스로 저장
  input_word = brand_name + search_word
  remove_words = [brand_name, search_word]

  #단어 빈도를 저장할 counter 객체 생성
  word_counts = Counter()

  #모든 문장에 대해
  for sent in df_text:
    '''#명사 추출
    # words = [word for word, pos in sent if pos in ['Noun', 'Adjective']]'''
    #검색어 제외하고 단어 리스트에 저장
    for_count_words = [word for word in sent if word not in remove_words]
    
    #단어 빈도 계산 (검색어와 한 글자 단어는 제외) - 단어와 빈도수를 계속 for문 돌면서 추가(update)
    word_counts.update(word for word in for_count_words if word != input_word and len(word) > 1)

  #가장 많이 등장하는 단어 7개만 출력
  #상위 단어 저장할 리스트 생성
  most_word = [] 
  for word, count in word_counts.most_common(7):
    #단어와 빈도수를 튜플 형태로 저장
    most_word_tuple = (word, count) 
    most_word.append(most_word_tuple)

  #최종 결과로 상위 7개 단어의 단어와 빈도수가 저장된 리스트 반환
  return most_word

In [ ]:
#감성 분석 
#파일 로드 함수
def load_files(filepath):
  with open(filepath, 'r', encoding = 'utf-8') as file:
    words = file.read().splitlines()
  return words

#위 함수로 감성사전 불러오기
pos_words = load_files('/content/drive/MyDrive/Colab Notebooks/knu_pos.txt')
neg_words = load_files('/content/drive/MyDrive/Colab Notebooks/knu_neg.txt')
unknown_words = load_files('/content/drive/MyDrive/Colab Notebooks/knu_unknown.txt')

#감성 분석 함수 - 긍정, 부정, 중립어 수 세기 -> 반환
def emotion_analysis(input_text):
  positive_count = 0 #긍정어 수
  negative_count = 0 #부정어 수
  neutral_count = 0 #중립어 수
  total_count = 0 #전체 단어수

  #단어 개수 세기 -> count에 추가
  for e_text in input_text:
    total_count += len(e_text)
    for word in e_text:
      if word in pos_words:
        positive_count += 1
      elif word in neg_words:
        negative_count += 1
      elif word in unknown_words:
        neutral_count += 1

  return total_count, positive_count, negative_count, neutral_count

In [ ]:
#랜덤으로 문장 3개 출력하는 함수
def random_sent(org_df):
  random_sents = []
  random_rows = org_df['Description'].sample(n=3)
  for row in random_rows:
    random_sents.append(row)
  return random_sents

In [ ]:
#검색어 주변 상위 빈도수의 단어 추출 함수
def near_words(sent_list, brand_name):
  #단어 빈도수 계산 함수와 유사
  near_words = []
  for sent in sent_list:
    #다시 pos tagging
    words_pos = okt.pos(' '.join(sent))
    #명사와 형용사만 추출(동사 제외)
    words = [word for word, pos in words_pos if pos in ['Noun', 'Adjective']]

    if brand_name in words:
      #문장의 단어 리스트에서 검색어의 index를 찾음
      search_word_index = words.index(brand_name)
      #검색어 index 양쪽으로 5개의 단어만 새 리스트에 저장
      near_words += words[max(0, search_word_index-5):search_word_index]
      near_words += words[search_word_index+1:min(len(words), search_word_index+6)]
  #counter 객체 생성
  word_counts = Counter()
  #near_words에서 단어 빈도수 계산
  word_counts.update(word for word in near_words if word != brand_name)
  #가장 상위의 단어 4개 따로 저장
  near_count_list = word_counts.most_common(4)

  return near_count_list

데이터 처리

In [ ]:
#브랜드 정보 검색 함수
def brand_inform(brand_name, search_word):
  #검색어 만들기
  search_key = brand_name + ' ' + search_word
  print()
  print('검색어:', search_key)

  #네이버 카페 데이터 처리
  i_cafe_df = naver_cafe(search_key) #검색 결과 dataframe으로 저장
  i_cafe_text = cafe_process_api(i_cafe_df) #검색 결과 전처리, 리스트로 저장
  i_cafe_most_word = word_count(i_cafe_text, brand_name, search_word) #빈도수 count -> 결과는 리스트[(단어, 빈도수)]
  cafe_near_count = near_words(i_cafe_text, brand_name) #brand name 주변 상위 빈도수 단어 count -> 결과는 리스트[(단어, 빈도수)]

  #결과 출력 부분
  print()
  print('카페글 분석 결과')
  print('가장 많이 등장하는 단어 7개: \n1위-', *i_cafe_most_word[0], '\n2위-', *i_cafe_most_word[1], '\n3위-', *i_cafe_most_word[2], '\n4위-', *i_cafe_most_word[3], '\n5위-', *i_cafe_most_word[4], '\n6위-', *i_cafe_most_word[5], '\n7위-', *i_cafe_most_word[6])

  print()
  print('검색 결과 중 브랜드명과 함께 검색된 상위의 단어입니다.')
  print(*cafe_near_count[0], sep=', ')
  print(*cafe_near_count[1], sep=', ')
  print(*cafe_near_count[2], sep=', ')

  print()
  print('-'*80)

  #네이버 뉴스 데이터 처리
  i_news_df = naver_news(search_key)
  i_news_text = news_process_api(i_news_df)
  i_news_most_word = word_count(i_news_text, brand_name, search_word)
  news_near_count = near_words(i_news_text, brand_name)

  #결과 출력 부분
  print()
  print('뉴스글 분석 결과')
  print('가장 많이 등장하는 단어 7개: \n1위-', *i_news_most_word[0], '\n2위-', *i_news_most_word[1], '\n3위-', *i_news_most_word[2], '\n4위-', *i_news_most_word[3], '\n5위-', *i_news_most_word[4], '\n6위-', *i_news_most_word[5], '\n7위-', *i_news_most_word[6])

  print()
  print("검색어와 가장 많이 같이 등장한 단어입니다. ")
  print(*news_near_count[0], sep=', ')
  print(*news_near_count[1], sep=', ')
  print(*news_near_count[2], sep=', ')
  
  print()

In [ ]:
def review(brand_name, review_str):
  #검색어 만들기 브랜드명+후기
  search_word = brand_name + ' ' + review_str
  print()
  print('검색어:', search_word)

  #네이버 카페 데이터 처리
  r_cafe_df = naver_cafe(search_word) #검색 결과 dataframe으로 저장
  r_cafe_text = cafe_process_api(r_cafe_df) #검색 결과 전처리, 리스트로 저장
  r_cafe_most_word = word_count(r_cafe_text, brand_name, review_str) #빈도수 count -> 결과는 리스트[(단어, 빈도수)]
  cafe_total_word, cafe_pos_word, cafe_neg_word, cafe_neu_word = emotion_analysis(r_cafe_text) #감성 분석 -> 전체 단어 개수, 긍정어, 부정어, 중립어 개수 변수에 저장
  cafe_random_sents = random_sent(r_cafe_df) #random row  리스트로 저장

  #결과 출력 부분
  print()
  print('후기 - 카페글 분석 결과')
  print('가장 많이 등장하는 단어 7개: \n1위-', *r_cafe_most_word[0], '\n2위-', *r_cafe_most_word[1], '\n3위-', *r_cafe_most_word[2], '\n4위-', *r_cafe_most_word[3], '\n5위-', *r_cafe_most_word[4], '\n6위-', *r_cafe_most_word[5], '\n7위-', *r_cafe_most_word[6])

  print()
  print(f'총 단어 개수: {cafe_total_word}')
  print(f'긍정어 개수: {cafe_pos_word}')
  print(f'부정어 개수: {cafe_neg_word}')
  print(f'중립어 개수: {cafe_neu_word}')

  print()
  print('검색 결과 예시')
  for row in cafe_random_sents:
    print(row)
  
  print()
  print('-'*80)

  #네이버 블로그 데이터 처리
  r_blog_df = naver_blog(search_word)
  r_blog_text = blog_process_api(r_blog_df)
  r_blog_most_word = word_count(r_blog_text, brand_name, review_str)
  blog_total_word, blog_pos_word, blog_neg_word, blog_neu_word = emotion_analysis(r_blog_text)
  blog_random_sents = random_sent(r_blog_df)

  #결과 출력 부분
  print()
  print('후기 - 블로그글 분석 결과')
  print('가장 많이 등장하는 단어 7개: \n1위-', *r_blog_most_word[0], '\n2위-', *r_blog_most_word[1], '\n3위-', *r_blog_most_word[2], '\n4위-', *r_blog_most_word[3], '\n5위-', *r_blog_most_word[4], '\n6위-', *r_blog_most_word[5], '\n7위-', *r_blog_most_word[6])

  print()
  print(f'총 단어 개수: {blog_total_word}')
  print(f'긍정어 개수: {blog_pos_word}')
  print(f'부정어 개수: {blog_neg_word}')
  print(f'중립어 개수: {blog_neu_word}')

  print()
  print('검색 결과 예시')
  for row in blog_random_sents:
    print(row)
  
  print()

Main 서비스 (사용자 input, output 코드)

In [ ]:
#사용자 입출력 cell

print('안녕하세요! \n자동차에 대한 정보를 주는 서비스입니다.')
print()

#프로그램 종료 전까지 계속 반복해서 돌아감
while True: 
  #정보와 후기 중 궁금한 것 입력받기
  category = input("무엇이 궁금하신가요? (정보 / 후기) - 종료 시 '종료' 입력: ")
  print()

  if category == '정보':
    #검색할 자동차 브랜드 입력받기
    brand_name = input('궁금한 자동차 브랜드를 입력해 주세요. : ')
    #사용자로부터 검색할 검색어 입력받기
    search_word = input('어떤 정보에 대해 검색하시나요? (그냥 브랜드만 궁금하다면 space를 한 번 눌러주세요): ')
    #브랜드 정보 검색어를 입력하여 처리하는 함수 불러오기
    brand_inform(brand_name, search_word)

  elif category == '후기':
    #검색할 자동차 브랜드 입력받기
    brand_name = input('궁금한 자동차 브랜드를 입력해 주세요. : ')
    review_str = '후기'
    #후기 검색어 입력하여 처리하는 함수
    review(brand_name, review_str)

  #종료할 시에는 종료 입력
  elif category == '종료':
    print('이용해 주셔서 감사합니다.')
    break
  
  else: 
    print('다시 입력해 주세요.')
    print()

  
  print('='*80)
  print()

안녕하세요! 
자동차에 대한 정보를 주는 서비스입니다.



검색어: 테슬라  

카페글 분석 결과
가장 많이 등장하는 단어 7개: 
1위- 모델 872 
2위- 판매 496 
3위- 가격 389 
4위- 전기차 291 
5위- 차량 199 
6위- 거래 174 
7위- 개인 170

검색 결과 중 브랜드명과 함께 검색된 상위의 단어입니다.
모델, 600
전기차, 177
가격, 170

--------------------------------------------------------------------------------

뉴스글 분석 결과
가장 많이 등장하는 단어 7개: 
1위- 주가 536 
2위- 전기차 489 
3위- 미국 466 
4위- 상승 458 
5위- 했다 339 
6위- 모델 239 
7위- 중국 237

검색어와 가장 많이 같이 등장한 단어입니다. 
주가, 279
전기차, 250
미국, 209


